In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

<h1>XGBoost Cloud Prediction Invocation Template</h1>
<h4>Invoke SageMaker Prediction Service</h4>

In [2]:
import boto3
import re
from sagemaker import get_execution_role
import sagemaker

In [3]:
# Acquire a realtime endpoint
endpoint_name = 'xgboost-ashrae-v5'
predictor = sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name)

In [4]:
from sagemaker.predictor import csv_serializer, json_deserializer

predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

In [5]:
df_all = pd.read_csv('ashrae_test.csv')

In [6]:
df_all.head()

,row_id,meter,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,hour,weekend
0,0,0,0,8.913685,17.8,4.0,11.7,0.0,0,6
1,1,0,0,7.908755,17.8,4.0,11.7,0.0,0,6
2,2,0,0,8.589886,17.8,4.0,11.7,0.0,0,6
3,3,0,0,10.072639,17.8,4.0,11.7,0.0,0,6
4,4,0,0,11.666573,17.8,4.0,11.7,0.0,0,6


In [7]:
df_all.columns[1:]

Index(['meter', 'primary_use', 'square_feet', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'hour',
       'weekend'],
      dtype='object')

In [8]:
# Need to pass an array to the prediction
arr_test = df_all[df_all.columns[1:]].values

In [9]:
type(arr_test)

numpy.ndarray

In [10]:
# For large number of predictions, we can split the input data and
# Query the prediction service.
# array_split is convenient to specify how many splits are needed
predictions = []
for arr in np.array_split(arr_test,10):
    result = predictor.predict(arr)
    result = result.decode("utf-8")
    result = result.split(',')
    predictions += [float(r) for r in result]

In [11]:
np.expm1(predictions)

array([ 1.23547699,  0.59316564,  0.38743032, ..., 26.20003136,
       13.29991061, 25.45951138])

In [12]:
df_all['meter_reading'] = np.expm1(predictions)

In [13]:
df_all.head()

,row_id,meter,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,hour,weekend,meter_reading
0,0,0,0,8.913685,17.8,4.0,11.7,0.0,0,6,1.235477
1,1,0,0,7.908755,17.8,4.0,11.7,0.0,0,6,0.593166
2,2,0,0,8.589886,17.8,4.0,11.7,0.0,0,6,0.387430
3,3,0,0,10.072639,17.8,4.0,11.7,0.0,0,6,2.818879
4,4,0,0,11.666573,17.8,4.0,11.7,0.0,0,6,13.612051


In [14]:
df_all[['row_id', 'meter_reading']].to_csv('predicted_meter_reading.csv',index=False)